# Latest Product Table

- p_id from **1 to 100** already present in  main table, it can be updated and deleted
- p_id from **101 to 200** can be inserted in to main table and after inserting it into main table it could also be deleted


### Libraries Used

In [1]:
import datetime
import os

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *


### Read the Main Table

In [2]:
# Read from main_table
Table = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main_table_new.csv")

# Add record type to main table
Table_new = Table.withColumn('record_type', lit("A"))

# Write in main_table folder
path = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
try:
    Table_new.write.format('csv').save(
        os.path.join(path, 'main_table'), header=True)
except:
    Table_new.write.mode('overwrite').format('csv').save(
        os.path.join(path, 'main_table'), header=True)


In [3]:
# GFG
class my_dictionary(dict):

    # __init__ function
    def __init__(self):
        self = dict()

    # Function to add key:value
    def add(self, key, value):
        self[key] = value

### Main CODE

In [4]:
# Storing all the "p_id as value" in a dictionary with "date as key", which we have inserted,deleted & updated            
total_products_updated = my_dictionary()
total_products_inserted = my_dictionary()
total_products_deleted = my_dictionary()

# Storing all the p_id in a list which we have deleted and inserted
total_products_inserted_list = []
total_products_deleted_list = []

# ###################################################################################################

for i in range(10, 0, -1):

    data_schema = [
        StructField('p_id', IntegerType(), True),
        StructField('p_name', StringType(), True),
        StructField('price', IntegerType(), True),
        StructField('date_timestamp', TimestampType(), True),
        StructField('record_type', StringType(), True)
    ]
    # Schema which we are accepting
    final_struc = StructType(fields=data_schema)

    # In this try-catch block we are only taking the previous day data, but initially there is
    # no previous day, therefore we are taking main table at begining
    try:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", t)
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass

        mainTable = spark.read.csv(path1, header=True, schema=final_struc)
    except:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", "main_table")
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass
        mainTable = spark.read.csv(path1, header=True, schema=final_struc)

    print("\n File read from --")
    print(path1, end='\n')

    # Address to the product_info folder
    address = "/home/bluepi/Downloads/Update/product_info/"
    previous_day = (datetime.datetime.today() -
                    datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    print("\n\t\ Date ----> "+previous_day+"\n")

    # before_insert = str(mainTable.count())
    print("Main_Table Count ----> " + str(mainTable.count()))

    # Address to the Previous Day folder
    new_address = address + previous_day
#     print("\nNew Address to read the folder ---->"+new_address)

# ###################################################################################################

    # Read the Previous Day folder
    per_day_data = spark.read.format('csv') \
        .options(header=True, inferschema=True) \
        .load(new_address)

    # Get the list of p_id which we have to inserted in the main table
    to_be_inserted = per_day_data.filter("record_type == 'I' ").collect()
    p_id_list_I = [i.p_id for i in to_be_inserted]
    
    # Since we don't want our Inserted product to be Inserted again
    for i in total_products_inserted_list :
        if i in p_id_list_I :
            print("\n{} is going to be inserted again\n".format(i))
            # Therefore removing already inserted products
            p_id_list_I.remove(i)
    # Storing all the p_id in a list which we have inserted
    total_products_inserted_list.extend(p_id_list_I)

    # Directly append new Inserted products
    mainTable_I_inserted = mainTable.union(
        per_day_data.filter("record_type == 'I' "))

    after_insert = str(mainTable_I_inserted.count())
    total_insert = str(per_day_data.filter("record_type == 'I' ").count())

    print("Total \"Inserted I\"----> " + total_insert)
    print("After Inserting----> " + after_insert)

    # Storing all the "p_id as value" in a dictionary with "date as key", which we have inserted
    total_products_inserted.add(previous_day, p_id_list_I)
    p_id_list_I.sort()
    print(" ".join(map(str, p_id_list_I)))
    print("\n")

# ###################################################################################################

# Created a new DataFrame of records to be updated
    from_per_day_data_U = per_day_data.filter("record_type == 'U' ")
    
# Get the list of p_id which we have to update taken from dated folders
    from_per_day_data_U_list = from_per_day_data_U.select("p_id").collect()

# List comprehension
    p_id_list_U = [i.p_id for i in from_per_day_data_U_list]
    p_id_list_U.sort()
    print("TO Be Update\"per_day_data\"")
    print(" ".join(map(str, p_id_list_U)))
#     print(p_id_list_U)

    # Storing all the "p_id as value" in a dictionary with "date as key", which we have updated
    total_products_updated.add(previous_day, p_id_list_U)

    total_update = str(len((p_id_list_U)))
    print("Update Count ------>"+total_update)
    print("\n")
    
# Get the products which we have to update, present in Main Table
    from_mainTable_U = mainTable_I_inserted.filter(
        col('p_id').isin(p_id_list_U))

    # Performed Union operation on the above tables
    mT_and_pDD_union = from_mainTable_U.union(from_per_day_data_U)
#     mT_and_pDD_union.orderBy(mT_and_pDD_union.p_id).show()

    # Performed GroupBy operation on P_ID and take the latest date only
    #    Rename the columns
    for_join_mT_and_pDD = mT_and_pDD_union.groupBy("p_id").agg(
        {"date_timestamp": "max"}).withColumnRenamed("max(date_timestamp)", "date_timestamp_1")
    for_join_mT_and_pDD = for_join_mT_and_pDD.withColumnRenamed(
        "p_id", "p_id_1")
#     for_join_mT_and_pDD.show()

    # Performed Join operation to pick only latest updates only
    joined = mT_and_pDD_union.join(for_join_mT_and_pDD, (
        mT_and_pDD_union.p_id == for_join_mT_and_pDD.p_id_1) & (
            mT_and_pDD_union.date_timestamp == for_join_mT_and_pDD.date_timestamp_1), 'inner')

    joined = joined.select(
        ['p_id', 'p_name', 'price', 'date_timestamp', 'record_type'])
#     joined.count()

    # First remove the p_id from Main_Table which we have to updated
    mainTable_U_updated = mainTable_I_inserted.filter(
        ~col('p_id').isin(p_id_list_U))
#     mainTable_U_updated.orderBy("p_id").count()

    # Then Add the Updated P_ID to the Main_Table
    mainTable_U_updated_new = mainTable_U_updated.union(joined)
#     after_update = str(mainTable_U_updated_new.orderBy("p_id").count())
#     mainTable_U_updated_new.orderBy("p_id").count()

# ###################################################################################################

    # Get the list of p_id which we have to delete
    to_be_deleted = per_day_data.filter("record_type == 'D' ").collect()
    p_id_list_D = [i.p_id for i in to_be_deleted]
    
    # Since we don't want our deleted product to be deleted again
    for i in total_products_deleted_list :
        if i in p_id_list_D :
            print("\n{} is going to be deleted again\n".format(i))
            # Therefore removing already deleted products
            p_id_list_D.remove(i)
    # Storing all the p_id in a list which we have deleted            
    total_products_deleted_list.extend(p_id_list_D)
            
    
    # Storing all the "p_id as value" in a dictionary with "date as key", which we have deleted
    total_products_deleted.add(previous_day, p_id_list_D)
    

#     print("\nList of p_id which we have to deleted taken from \"per_day_data\"")
#     print(p_id_list_D)
    total_deleted = str(len((p_id_list_D)))
    p_id_list_D.sort()
    print("\nTotal Deleted ------>"+str(len((p_id_list_D))))
    print(" ".join(map(str, p_id_list_D)))

    # Remove the deleted p_id from main_table
    mainTable_D_deleted = mainTable_U_updated_new.filter(
        ~col('p_id').isin(p_id_list_D))

    after_delete = str(mainTable_D_deleted.count())
    print("After Deleting Count---->"+after_delete)
    

# ###################################################################################################

    # This writes the DF in different files becaues of parallism
    t = "main_table"+str(previous_day)
#     print(t,end="\n")

#     try:
    mainTable_D_deleted.write.format('csv').save(
            os.path.join(path, t), header=True)
#         print("\n Main Table Stored (overwritten-NO)")
#     except:
#         mainTable_D_deleted.write.mode('overwrite').format(
#             'csv').save(os.path.join(path, t), header=True)
#         print("\n Main Table Stored (overwritten-YES)")

    print("\n Main Table Stored ")
    print(os.path.join(path, t))

    print("\n***************************************************************************************\n")
#     input("Press Enter to continue...")


 File read from --
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table

	\ Date ----> 31-03-2020

Main_Table Count ----> 100
Total "Inserted I"----> 9
After Inserting----> 109
121 126 130 139 149 165 174 176 192


TO Be Update"per_day_data"
26 30 32 36 40 42 45 45 55 56 58 68 86
Update Count ------>13



Total Deleted ------>9
5 8 15 17 22 43 85 87 93
After Deleting Count---->100

 Main Table Stored 
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table31-03-2020

***************************************************************************************


 File read from --
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table31-03-2020

	\ Date ----> 01-04-2020

Main_Table Count ----> 100
Total "Inserted I"----> 10
After Inserting----> 110
109 122 152 156 161 170 172 188 198 200


TO Be Update"per_day_data"
13 13 18 38 48 52 59 67 79 80 81 81 82 90 90 90 96 99
Update Count ------>18



Total Deleted ------>9
2 16 29 49 74 77 84 89 

### Products Updated more than once in one day

In [5]:
l = mT_and_pDD_union.groupBy("p_id").agg({'p_id':'count'}).filter("count(p_id)>1").select('p_id').collect()
l1 = [ i.p_id for i in l]
mT_and_pDD_union.filter( col('p_id').isin(l1) ).orderBy(col('p_id')).show()

+----+----------+-----+-------------------+-----------+
|p_id|    p_name|price|     date_timestamp|record_type|
+----+----------+-----+-------------------+-----------+
|  21|Trippledex|  277|2020-04-03 09:28:03|          U|
|  21|Trippledex|  618|2020-04-08 15:42:48|          U|
|  23|     Subin|  213|2020-04-08 22:00:47|          U|
|  23|     Subin|  408|2020-01-27 15:36:03|          A|
|  27|   Konklux| 1389|2020-04-07 15:30:37|          U|
|  27|   Konklux| 1079|2020-04-08 15:58:38|          U|
|  35| Cardguard|  583|2020-04-08 03:24:19|          U|
|  35| Cardguard|  686|2020-04-05 09:27:10|          U|
|  54| Voyatouch|  232|2020-04-04 03:34:25|          U|
|  54| Voyatouch| 1282|2020-04-08 21:25:24|          U|
|  56| Voyatouch|  330|2020-04-08 03:21:29|          U|
|  56| Voyatouch|  253|2020-03-30 21:34:20|          U|
|  70|Voltsillam|  283|2020-04-05 15:49:35|          U|
|  70|Voltsillam|  710|2020-04-08 03:45:08|          U|
|  78|        It| 1252|2020-04-04 15:41:26|     

### Final Main Table

In [6]:
mainTable_D_deleted.filter( col('p_id').isin(l1) ).orderBy(col('p_id')).show()

+----+-----------+-----+-------------------+-----------+
|p_id|     p_name|price|     date_timestamp|record_type|
+----+-----------+-----+-------------------+-----------+
|  21| Trippledex|  618|2020-04-08 15:42:48|          U|
|  23|      Subin|  213|2020-04-08 22:00:47|          U|
|  27|    Konklux| 1079|2020-04-08 15:58:38|          U|
|  35|  Cardguard|  583|2020-04-08 03:24:19|          U|
|  56|  Voyatouch|  330|2020-04-08 03:21:29|          U|
|  70| Voltsillam|  710|2020-04-08 03:45:08|          U|
|  78|         It| 1030|2020-04-08 09:54:23|          U|
| 101|     Bigtax|  369|2020-04-08 03:35:30|          U|
| 105|   Tres-Zap|  407|2020-04-08 15:56:55|          U|
| 106|     Vagram|  909|2020-04-08 15:45:31|          U|
| 109|  Holdlamis|  610|2020-04-08 03:41:22|          U|
| 136|        Job| 1258|2020-04-08 15:50:41|          U|
| 138|    Cardify| 1342|2020-04-08 09:53:58|          U|
| 143|       Span| 1497|2020-04-08 15:22:24|          U|
| 151|    Sonsing|  457|2020-04

In [7]:
import pprint
# prints the formatted representation of PrettyPrinter object
pp = pprint.PrettyPrinter(indent=4)

for i in range(10,0,-1) :
    previous_day = (datetime.datetime.today() - datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    pp.pprint(previous_day)
    print("inserted :\t")
    print(' '.join(map(str, total_products_inserted.get(previous_day))))
#     pp.pprint(total_products_inserted.get(previous_day))
    print("updated :\t")
    print(' '.join(map(str, total_products_updated.get(previous_day))))
#     pp.pprint(total_products_updated.get(previous_day))
    print("deleted :\t")
    print(' '.join(map(str, total_products_deleted.get(previous_day))))
#     pp.pprint(total_products_deleted.get(previous_day))
    print("\n")

'31-03-2020'
inserted :	
121 126 130 139 149 165 174 176 192
updated :	
26 30 32 36 40 42 45 45 55 56 58 68 86
deleted :	
5 8 15 17 22 43 85 87 93


'01-04-2020'
inserted :	
109 122 152 156 161 170 172 188 198 200
updated :	
13 13 18 38 48 52 59 67 79 80 81 81 82 90 90 90 96 99
deleted :	
2 16 29 49 74 77 84 89 139


'02-04-2020'
inserted :	
124 131 141 157 163 164 178 189 195
updated :	
6 12 18 20 24 28 35 35 36 44 45 61 65 73 81 126 156 161 174 174 178 188 189 195
deleted :	
198


'03-04-2020'
inserted :	
103 105 137 146 160 162 175 199
updated :	
1 28 30 31 31 32 37 40 41 45 57 60 60 94 124 126 162 164 176 178 195
deleted :	
57 76 94 156 172


'04-04-2020'
inserted :	
115 117 136 148 155 173 183 197
updated :	
21 21 69 70 72 73 149 157 161 188 199
deleted :	
1 6 9 18 67 79 81 174


'05-04-2020'
inserted :	
112 116 125 158 167 171 179 190 196
updated :	
20 25 26 36 38 52 54 58 58 64 65 69 70 71 73 78 78 95 109 122 148 183 189 196 200
deleted :	
13 24 68 73 146 146 155


'06-04-2020'


In [8]:
total_products_deleted_list.sort()
print(' '.join(map(str,total_products_deleted_list)))

1 2 3 4 5 6 8 9 13 15 16 17 18 22 24 26 29 30 34 37 43 49 52 54 57 59 63 66 67 68 73 74 76 77 79 81 83 84 85 87 88 89 93 94 96 97 100 103 115 122 125 128 135 139 146 146 149 155 156 158 161 171 172 174 176 188 190 192 198


In [9]:
total_products_inserted_list.sort()
print(' '.join(map(str,total_products_inserted_list)))

101 102 103 104 105 106 107 108 109 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 133 135 136 137 138 139 141 142 142 143 145 146 147 148 149 150 151 152 153 154 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 186 187 188 189 190 191 192 194 194 195 196 197 198 199 200
